Using multiple types of solvers for DMET fragments
======

This notebook demonstrates how to use a different electronic structure solver for each framgent that is produced by the DMET problem decomposition. For details and theory about DMET, see the other examples provided with the documentation.

We set up the molecule that we use in this example.

In [ ]:
import py3Dmol

H8_He2 = """
HE         1.6180339887          0.0000000000          0.0000000000
HE         1.3090169944          0.9510565163          0.0000000000
H          0.5000000000          1.5388417686          0.0000000000
H         -0.5000000000          1.5388417686          0.0000000000
H         -1.3090169944          0.9510565163          0.0000000000
H         -1.6180339887          0.0000000000          0.0000000000
H         -1.3090169944         -0.9510565163          0.0000000000
H         -0.5000000000         -1.5388417686          0.0000000000
H          0.5000000000         -1.5388417686          0.0000000000
H          1.3090169944         -0.9510565163          0.0000000000
"""
view = py3Dmol.view(width=400,height=400)
view.addModel("10\n" + H8_He2,'xyz',{'keepH': True})
view.setStyle({'sphere':{}})
view.setStyle({'model':0},{'sphere':{'colorscheme':'cyanCarbon','scale':'0.2'}})
view.zoomTo()
view.show()

from pyscf import gto
mol = gto.Mole() # Instantiate the molecule class in PySCF
mol.atom = H8_He2   # The coordinates of the atoms of the 10-hydrogen-atom ring are defined above
mol.basis = "3-21g" # Use "3-12g" as the basis set
mol.charge = 0 # Assign the charge of the molecule 
mol.spin = 0 # Assign the spin of the molecule
mol.build() # Build the molecule object

As usual, we begin with importing the modules from OpenQEMIST. 

In [ ]:
from openqemist import electronic_structure_solvers as ess
from openqemist import problem_decomposition as pd

The DMET object can be used in two ways. The default behaviour of the solver is to use an instance of an electronic structure solver to solve all the fragments that are produced by the decomposition. This is shown below:

In [ ]:
dmet = pd.DMETProblemDecomposition()
dmet.electronic_structure_solver = ess.CCSDSolver()

energy = dmet.simulate(mol, [2,2,2,2,2])

print("DMET energy with CCSD is ", energy)

The DMET object can also use a different electronic structure solver to solve each fragment. This is done with the optional `fragment_solvers` parameter. The value passed here should be a list of `ElectronicStructureSolver` instances that has as many elements as there are fragments. This is shown below.

In [ ]:
# Create instances of the solvers that we want to use
fci = ess.FCISolver()
from openqemist import quantum_solvers as qs
vqe = ess.VQESolver()
vqe.hardware_backend_type = qs.MicrosoftQSharpParametricSolver
vqe.ansatz_type = qs.MicrosoftQSharpParametricSolver.Ansatze.UCCSD

# Use the VQE sovler to solve two helium fragments and the FCI solver for the hydrogen
solvers = [vqe, vqe] + [fci for i in range(8)]
energy = dmet.simulate(mol, [1,1,1,1,1,1,1,1,1,1], fragment_solvers=solvers)

print("Mixed solver energy is ", energy)